*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  ###!pip install tf-nightly
  print("Aleardy done !")
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
###!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [29]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = f".\..\data\\fcc_sms_text_classification\train-data.tsv"
test_file_path = f".\..\data\\fcc_sms_text_classification\valid-data.tsv"

In [81]:
train_data= pd.read_csv("../data/fcc_sms_text_classification/train-data.tsv", on_bad_lines='skip',sep=";;;",engine='python')
test_data=pd.read_csv("../data/fcc_sms_text_classification/valid-data.tsv", on_bad_lines='skip',sep=";;;",engine='python')

In [82]:
train_data.head()

,sms_class,message
0,ham,ahhhh...just woken up!had a bad dream about u...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like!..."
3,ham,mum say we wan to go then go... then she can ...
4,ham,never y lei... i v lazy... got wat? dat day ü...


In [84]:
df_train=train_data[["message","sms_class"]]
df_test=test_data[["message","sms_class"]]

In [88]:
df_test.head()

,message,sms_class
0,i am in hospital da. . i will return home in e...,ham
1,"not much, just some textin'. how bout you?",ham
2,i probably won't eat at all today. i think i'm...,ham
3,don‘t give a flying monkeys wot they think and...,ham
4,who are you seeing?,ham


### ⚙️ Prepare labels for training

In [89]:
df_train["sms_class"] = pd.factorize(df_train["sms_class"])[0]
df_test["sms_class"] = pd.factorize(df_test["sms_class"])[0]

In [91]:
df_train.head()

,message,sms_class
0,ahhhh...just woken up!had a bad dream about u...,0
1,you can never do nothing,0
2,"now u sound like manky scouse boy steve,like!...",0
3,mum say we wan to go then go... then she can ...,0
4,never y lei... i v lazy... got wat? dat day ü...,0


👇 the step below is important. It's not only about batching the data sets, but also reshape it to make it works when fitting the model. Otherwise, we will get incompatible layers errors: expecting ndim=3, got ndim=2.

In [92]:
train_labels =  df_train["sms_class"].values
train_ds = tf.data.Dataset.from_tensor_slices(
    (df_train["sms_class"].values, train_labels)
)

In [93]:
test_labels =  df_test["sms_class"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (df_test["sms_class"].values, test_labels)
)
test_ds.element_spec

(TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [94]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

🛠️ Create a TextVectorization layer for our model

In [97]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
vec = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

Let's show the vocabulary that our vectorizer has learned.

In [102]:
vocab = np.array(vec.get_vocabulary())
vocab[:20]

array(['', '[UNK]'], dtype='<U5')

Create the model


In [103]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

Train the model against our data sets.



In [104]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'sequential/Cast' defined at (most recent call last):
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 1

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
